# Gaining familiarity with web3py

In this notebook I will work through the examples provided in the [web3py documentation](https://web3py.readthedocs.io/en/stable/examples.html) to get a better feeling for the functionality. The goal is to be able to compile, deploy and interact with Solidity contracts directly from within Python. This is important since we need this functionality to be able to dynamically dispatch and interact with contracts from within the server backend of the LUCE technical prototype.  

**Update:** Since the old way to deploy contracts via `w3.eth.contract.deploy()` is deprecated this notebook also contains experiments with the new contract creation methods as outlined in the contract section of the web3.py documentation [here](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=abi).

### Software used

Over the last weeks I developed LuceVM, an Ubuntu 16.04 virtual machine that contains all required software and servers. Using the VM allows for a reproducible development environment and abstracts away the layer of managing dependencies for the different packages. It also makes it easier to ensure all components interact correctly with each other.  

The most important tools for us at this stage are:
* Python 3.7
    * web3
    * solc-py-x
* Jupyter Notebook Server for interaction and documentation
* Virtual Environments via conda to handle dependencies
* Solidity Compiler
* Node.js
* Ganache

### Preparation: Connect Python to local Ganache Node

First we start an instance of Ganache, either via GUI or the  ```run_ganache.sh``` script in LuceVM.  
Update: Running Ganache is handled via `run_servers_tmux.sh` directly as part of the VM setup process now.

In [1]:
from web3 import Web3, HTTPProvider

# Ganache Connection
w3 = Web3(Web3.WebsocketProvider("wss://rinkeby.infura.io/ws/v3/839112f3db884bde86889ebbac153ced"))
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

Alternatively we can also connect to Ganache running on the host machine:  
(Change settings to allow incoming connections from 192.168.72.1 or 0.0.0.0)

In [2]:
w3.isConnected()

True

### Obtain Accounts

In [ ]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [ ]:
# Display all 10 default accounts
accounts

In [ ]:
# Display address of first account
accounts[0]

Store credentials of first account for testing:

In [ ]:
# Wallet address
wallet_address       = "0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA"

# Private key (from Ganache interface)
wallet_private_key   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"

### Send ether

In [ ]:
# Define a function to conveniently send ether
import time
def send_ether(amount_in_ether, recipient_address):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    # How many transactions have been made by wallet?
    nonce = w3.eth.getTransactionCount(wallet_address)
    
    # Specify transcation details
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, wallet_private_key)

    # Send transaction & store transaction hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    # Check if transaction was added to blockchain
    time.sleep(0.5)
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    return txn_hash

In [ ]:
# Set recipient
recipient = accounts[1]

In [ ]:
# Send ether and store transaction hash
txn_hash = send_ether(1.5,recipient)

In [ ]:
# Count transactions performed by a given address
w3.eth.getTransactionCount(wallet_address)

### Check Balance

There are two accounts we are interested in:  
The sender: `wallet_address`  
And the recipient: `recpient`

In [ ]:
# Check balance
w3.eth.getBalance(recipient)

In [ ]:
# Balance in ether
w3.fromWei(w3.eth.getBalance(recipient), 'ether')

In [ ]:
# Balance of our primary account
w3.fromWei(w3.eth.getBalance(wallet_address), 'ether')

In [ ]:
tx_hash = send_ether(1, "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E")

In [ ]:
w3.fromWei(w3.eth.getBalance("0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"), 'ether')

### Look up a block

In [ ]:
w3.eth.getBlock(1)

In [ ]:
# Retrieve the last block
w3.eth.getBlock('latest')

In [ ]:
# Directly obtain the current block number
w3.eth.blockNumber

### Loop up a transaction via hash

In [ ]:
w3.eth.getTransaction(txn_hash)

In [ ]:
# Transaction receipt
w3.eth.getTransactionReceipt(txn_hash)

### Compile contract

Important that py-solc is installed for code to execute.  
Update: No need to worry anymore -> LuceVM has all correct dependencies installed

In [ ]:
#!{sys.executable} -m pip install py-solc

In [12]:
#!{sys.executable} -m pip install py-solc-x
solcx.install_solc('0.6.12')

Version('0.6.12')

In [6]:
import sys
import time
import pprint

from web3.providers.eth_tester import EthereumTesterProvider
from web3 import Web3
import solcx
from solcx import compile_source

def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()

    return compile_source(source, solc_version="0.6.2")

In [7]:
contract_source_path = './data/ConsentCode.sol'

In [8]:
compiled_sol = compile_source_file('./data/ConsentCode.sol')

In [9]:
contract_id, contract_interface = compiled_sol.popitem()

Up to this point everything is working. We can compile the contract and also have the abi and binary code conveniently stored in our ```contract_interface```.

### Deploy Contract

Method 1:

In [10]:
DEBUG = True
# Define default account (used as default for 'from' property in transaction dictionary)
private_key = "52417fb192c8cb46bf2b76e814992a803910d42cd19ca0ae0a83c5de97c6dbd6"
public_key = w3.eth.account.privateKeyToAccount(private_key).address
CHAIN_ID = 4


In [11]:
def sign_and_send(contract_txn, private_key, name):
    try:
        signed_txn = w3.eth.account.signTransaction(contract_txn, private_key)
      
        tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
      
    
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        transaction = receipt_to_dict(tx_receipt, name)
        
    except ValueError as e:
        if DEBUG:
            print()
            print(e)
        return [e,name]
    return transaction

def receipt_to_dict(tx_receipt, name):
    receipt = {}
    receipt["blockHash"] = tx_receipt.blockHash.hex()
    receipt["blockNumber"] = tx_receipt.blockNumber
    receipt["contractAddress"] = tx_receipt.contractAddress
    receipt["cumulativeGasUsed"] = tx_receipt.cumulativeGasUsed
    receipt["effectiveGasPrice"] = w3.toInt(hexstr = tx_receipt.effectiveGasPrice)
    receipt["from"] = tx_receipt["from"]
    receipt["gasUsed"] = tx_receipt.gasUsed
    receipt["logsBloom"] = tx_receipt.logsBloom.hex()
    receipt["status"] = tx_receipt.status
    receipt["to"] = tx_receipt.to
    receipt["transactionHash"] = tx_receipt.transactionHash.hex()
    receipt["transactionIndex"] = tx_receipt.transactionIndex
    receipt["type"] = tx_receipt.type
    receipt["fees"] =  receipt["effectiveGasPrice"] * receipt["gasUsed"]
    receipt["transaction name"] = name

    return receipt

In [12]:
# Store abi and bytecode of contract
abi = contract_interface['abi']
bytecode = contract_interface['bin']

In [20]:
# Create contract object
my_contract = w3.eth.contract(abi=abi,bytecode=bytecode)
nonce = w3.eth.getTransactionCount(public_key)
txn_dict = {
        'from': public_key,
        'chainId': CHAIN_ID,
        'gasPrice': w3.toWei('20', 'gwei'),
        'nonce': nonce,
        }
#gas = w3.eth.estimateGas(contract_txn)ù
gas = my_contract.constructor().estimateGas()

txn_dict["gas"]=gas*2

contract_txn = my_contract.constructor().buildTransaction(txn_dict)

tx_receipt = sign_and_send(contract_txn, private_key, "deployment of Test")


192


In [58]:
#contract_address = tx_receipt["contractAddress"]
contract_address =  "0x9495Fad019Ab2566de987ce20bEC0626532C5325"

In [60]:
contract_instance = w3.eth.contract(address=contract_address, abi=abi)
print(contract_instance)
bollarray = []

for i in range(0,49):
    bollarray.append(False)
    
contract_txn =  contract_instance.functions.CheckAccess(bollarray).call()
print(contract_txn)
#tx_receipt = sign_and_send(contract_txn, private_key, "change number")

False


In [84]:
contract_txn

False

In [ ]:
contract_instance = w3.eth.contract(address="0xd79Ee34C7d0DF71AFE54F59455BB25F3675A57ba", abi=abi)

contract_instance.functions.getNumber().call()

In [ ]:
# Deploy contract and obtain transaction hash
tx_hash = my_contract.deploy()

This does work however this approach is deprecated..  
Even though the web3.py documentation itself still uses this approach in their examples section.

Method 2:

In [ ]:
def deploy_contract(w3, contract_interface):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).deploy()

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address

In [ ]:
address = deploy_contract(w3, contract_interface)

In [ ]:
print("Deployed {0} to: {1}\n".format(contract_id, address))

This also works but again with deprecation warning.

### Interact with Contract

In [ ]:
# Create interface to deployed contract
deployed_contract = w3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

**Estimate gas usage**

In [ ]:
# Estimate gas usage for function call
gas_estimate = deployed_contract.functions.setVar(255).estimateGas()
print("Gas estimate to transact with setVar: {0}\n".format(gas_estimate))

In [ ]:
def wait_for_receipt(w3, tx_hash, poll_interval):
    while True:
        tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
        if tx_receipt:
            return tx_receipt
        time.sleep(poll_interval)

In [ ]:
# Execute function call to smart contract
if gas_estimate < 100000:
    print("Sending transaction to setVar(255)\n")
    tx_hash = deployed_contract.functions.setVar(255).transact()
    receipt = wait_for_receipt(w3, tx_hash, 1)
    print("Transaction receipt mined: \n")
    pprint.pprint(dict(receipt))
else:
    print("Gas cost exceeds 100000")

In [ ]:
# Call setVar on contract
deployed_contract.functions.setVar(4).transact()

In [ ]:
# Obtain value from getVar on contract
deployed_contract.functions.getVar().call()

### New Contract Deployment Approach
Since the old way to deploy contracts via `w3.eth.contract.deploy()` is deprecated I also try out another pathway, as outlined in the contract section of the web3.py documentation [here](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=abi).

Note: I replace solc with solcx since solc is throwing errors during compilation. solcx can compile the contract.  
Update: I am still using solcx instead of solc but LuceVM now already has all required packages installed

In [ ]:
#!{sys.executable} -m pip install -U web3[tester]

There is a requirement conflict with eth-abi when trying to install web3tester. See if we can use ganache as sandbox instead.

In [ ]:
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [ ]:
# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.21;

contract Greeter {
    string public greeting;

    function Greeter() public {
        greeting = 'Hello';
    }

    function setGreeting(string _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string) {
        return greeting;
    }
}
'''

In [ ]:
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Greeter']

The compilation works fine when using solcx instead of solc.

In [ ]:
# web3.py instance
# w3 = Web3(Web3.EthereumTesterProvider())
# This causes a requirement conflix with eth-abi.. Use Ganache instead

In [ ]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [ ]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

**Instantiate Contract Object**

In [ ]:
# Instantiate contract
Greeter = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

**Deploy Contract**

In [ ]:
# Submit the transaction that deploys the contract
tx_hash = Greeter.constructor().transact()

Halleluhjah! This worked :D  
The contract is deployed.  
And this time without a deprecation warning.  
So this is the new recommended approach of deployment.

In [ ]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [ ]:
tx_receipt

In [ ]:
# Create the contract instance with the newly-deployed address
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

**Read contract state**

In [ ]:
# Display the default greeting from the contract
print('Default contract greeting: {}'.format(
    greeter.functions.greet().call()
))

In [ ]:
greeter.functions.abi

In [ ]:
greeter.functions.greet().call()

**Change contract state**

In [ ]:
tx_hash = greeter.functions.setGreeting('new greeting').transact()

In [ ]:
# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

In [ ]:
greeter.functions.greet().call()

All is working as intended! I am very happy. This gives enough foundation. The examples here can be used as reference for how to interact with LUCE contract functions.

### Other Useful Snippets from web3 documentation

**web3.eth.*account* object**

In [ ]:
# Create a new account
test_account = w3.eth.account.create()

In [ ]:
# Display address
test_account.address

In [ ]:
# Show private key
test_account.privateKey

In [ ]:
# Restore account from private key
test_account_restored = w3.eth.account.privateKeyToAccount("0xd6d5a9f687f225d069c80a3da11aadfd4b3b937dba40c9a4343b7eab0b69d35d")

In [ ]:
# Check if indeed address of restored account is the same
test_account_restored.address == test_account.address

In [ ]:
type(test_account)

**Sign a transaction**

In [ ]:
# 1) First define a transaction dictionary
tx_dic ={
    'to': '0xF0109fC8DF283027b6285cc889F5aA624EaC1F55',
    'value': 1000000000,
    'gas': 2000000,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': w3.eth.getTransactionCount(test_account.address)
    
}

In [ ]:
# 2) Then sign
test_account.signTransaction(tx_dic)

---

**web3.eth.*contract* object**

In [ ]:
test_contract = w3.eth.contract()

In [ ]:
type(test_contract)

In [ ]:
test_contract.address

In [ ]:
test_contract.abi

In [ ]:
test_contract.bytecode

In [ ]:
test_contract.deploy

In [ ]:
test_function = test_contract.functions

In [ ]:
test_function

### Drop Things Here

In [ ]:
# Contract address from Ganache
contract_address     = "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"